In [792]:
import os

proj_dir = '/Users/nickbachelder/Desktop/Kaggle/Linemen'
os.chdir( os.path.join(proj_dir, 'code/objects') )

%run load_modules.ipynb
%run tracking_objects.ipynb

In [793]:
data_dir = os.path.join(proj_dir, 'data' )
os.chdir( proj_dir )
os.system("rm -rf {}".format( data_dir ))
os.system('mkdir data')
os.chdir( data_dir )
os.system("kaggle competitions download -c nfl-big-data-bowl-2023")
os.system( 'unzip {}.zip'.format( os.path.join(data_dir, 'nfl-big-data-bowl-2023')) )

curr_cd = os.getcwd()
files = [x for x in os.listdir(curr_cd) if os.path.isfile(os.path.join(curr_cd, x))]

player_files = [os.path.join(curr_cd, x) for x in files if x.endswith('players.csv')]
player_tracking_files = [os.path.join(curr_cd, x) for x in files if x.startswith('week')]
play_details_files = [os.path.join(curr_cd, x) for x in files if x.endswith('ScoutingData.csv')]
play_info_files = [os.path.join(curr_cd, x) for x in files if x.endswith('plays.csv')]


100%|██████████| 172M/172M [00:13<00:00, 13.3MB/s] 



Archive:  /Users/nickbachelder/Desktop/Kaggle/Linemen/data/nfl-big-data-bowl-2023.zip
  inflating: games.csv               
  inflating: pffScoutingData.csv     
  inflating: players.csv             
  inflating: plays.csv               
  inflating: week1.csv               
  inflating: week2.csv               
  inflating: week3.csv               
  inflating: week4.csv               
  inflating: week5.csv               
  inflating: week6.csv               
  inflating: week7.csv               
  inflating: week8.csv               


In [794]:
all_data = master_track(track_paths = player_tracking_files, 
                            play_info_path = play_info_files[0],
                            play_details_path = play_details_files[0],
                            players_path = player_files[0])

In [795]:
all_data.load_training_data(week = 3)

Getting week 1 training data


100%|██████████| 1170/1170 [05:50<00:00,  3.34it/s]


Getting week 2 training data


100%|██████████| 1062/1062 [05:29<00:00,  3.22it/s]


Done


In [796]:
all_data.get_averages_up_to_week(week = 3)

Getting week 1 average metrics by position and blocker number
Getting week 2 average metrics by position and blocker number
Done


In [797]:
# figure out where these issues are coming from later and fix

all_dat_labels = all_data.get_rush_sequences_labels(week = 2, normalize = True)

training_dat, test_dat = train_test_split(all_dat_labels, test_size = 0.2)

print(f'Length train : {len(training_dat)}')
print(f'Length test : {len(test_dat)}')

Length train : 936
Length test : 234


In [798]:
# model building

model = SequenceModel(n_features = training_dat[0][0].shape[1], n_classes = 2, n_hidden = 15, n_layers = 1)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.01)
train_dataset = RushDataset(sequences = training_dat, sequence_length=30)
train_loader = DataLoader(train_dataset, batch_size = 50, shuffle = True)
test_dataset = RushDataset(sequences = test_dat, sequence_length=30)
test_loader = DataLoader(test_dataset, batch_size = 50, shuffle = True)

lstm_model = train_rush_lstm(train_loader, test_loader, model, loss_function, optimizer, num_epochs = 20)

Epoch 0
---------
Overall Train loss: 0.6627775932613172 , Overall Train AUC: 0.5252436439234791
Train loss for period 1: 0.6648907881034049 , Train AUC for period 1: 0.5136135756148998
Train loss for period 2: 0.6641355251011095 , Train AUC for period 2: 0.5214295593320091
Train loss for period 3: 0.663748468223371 , Train AUC for period 3: 0.5233756590401923
Train loss for period 4: 0.6626910724137959 , Train AUC for period 4: 0.5270977709114986
Train loss for period 5: 0.6609214701150593 , Train AUC for period 5: 0.5341555212241594
Train loss for period 6: 0.6602782011032104 , Train AUC for period 6: 0.5379433531964981
Overall test loss: 0.6817781090736389 , Overall test AUC: 0.6025396766014051
Test loss for period 1: 0.6829615592956543 , Test AUC for period 1: 0.5303434343434343
Test loss for period 2: 0.6825774908065796 , Test AUC for period 2: 0.5658062102506547
Test loss for period 3: 0.6824111580848694 , Test AUC for period 3: 0.5851672278338945
Test loss for period 4: 0.681790

In [803]:
### plotting all below

play_id = 2021091209198

play_prob = predict_play(master_track = all_data, play_id = play_id, model = lstm_model, normalize = True)
player_metrics = get_all_player_metrics_on_play(master_track = all_data, play_id = play_id, model = lstm_model)


rusher_play_info = all_data.get_play(play_id).reset_index(drop = 1)
rusher_play_info.time_after_snap = round(rusher_play_info.time_after_snap, 1)

this = get_all_player_metrics_on_play(master_track = all_data, play_id = play_id, model = lstm_model)
player_metrics = []
for player_id in this.keys():
    metrics = this.get(player_id)
    metrics_df = pd.DataFrame({'nflId': float(player_id), 'rusher_metric' : metrics, 'time_after_snap' : np.array(range(0, len(metrics))) / 10})
    player_metrics.append(metrics_df)
player_metrics_df = pd.concat(player_metrics)

rusher_play_info =  rusher_play_info.merge(player_metrics_df, on = ['nflId', 'time_after_snap'], how = 'left')



def_match = all_data.play_details_df.loc[(all_data.play_details_df.playId == play_id) & (all_data.play_details_df.pff_nflIdBlockedPlayer.notna()), ['playId', 'nflId', 'pff_nflIdBlockedPlayer']]
defender_track = []
for defender in def_match.nflId:
    this_def = all_data.search_track_weeks(variables = ["playId", "nflId"], variable_values = [play_id, defender])
    defender_track.append(this_def)
defender_track = pd.concat(defender_track).merge(def_match, on = ['playId', 'nflId'], how = 'left').loc[:,['nflId', 'playId', 'time_after_snap', 'pff_nflIdBlockedPlayer', 'x', 'y']].rename(columns = {'pff_nflIdBlockedPlayer' : 'nflId', 'nflId' : 'blockerId', 'x' : 'x_block', 'y' : 'y_block'})

# get qb track
qb_track = all_data.get_qb_track_on_play(play_id).loc[:, ['playId', 'time_after_snap', 'x', 'y', 's', 'a']].rename(columns = {'x' : 'x_qb', 'y' : 'y_qb', 's' : 's_qb', 'a' : 'a_qb'})
defender_qb_track = defender_track.merge(qb_track, on = ['playId', 'time_after_snap'], how = 'left')
defender_qb_track.time_after_snap = round(defender_qb_track.time_after_snap, 1)

block_df = defender_qb_track.loc[(defender_qb_track.time_after_snap >= 0) & (defender_qb_track.time_after_snap < len(play_prob) / 10), ['blockerId', 'time_after_snap', 'x_block', 'y_block']].rename(columns = {'blockerId' : 'nflId', 'x_block' : 'x', 'y_block' : 'y'}).drop_duplicates()
rush_df = rusher_play_info.loc[(rusher_play_info.time_after_snap >= 0) & (rusher_play_info.time_after_snap < len(play_prob) / 10), ['nflId', 'time_after_snap', 'rusher_metric', 'x', 'y']].drop_duplicates()
qb_df = defender_qb_track.loc[(defender_qb_track.time_after_snap >= 0) & (defender_qb_track.time_after_snap < len(play_prob) / 10), ['time_after_snap', 'x_qb', 'y_qb']].rename(columns = {'x_qb' : 'x', 'y_qb' : 'y'}).drop_duplicates()
qb_df['nflId'] = 1


unique_time_after_snap = rush_df.time_after_snap.unique()
n_player_rush = len(rush_df.nflId.unique())
n_player_block = len(block_df.nflId.unique())

n_frame = len(unique_time_after_snap)
motion_rush = np.empty(shape = (n_frame,3,n_player_rush))
motion_block = np.empty(shape = (n_frame,3,n_player_block))
motion_qb = np.empty(shape = (n_frame,3, 1))
frame = 0
for time_after in unique_time_after_snap:
    this_frame_rush = rush_df.loc[rush_df.time_after_snap == time_after].copy()
    x_this_frame_rush, y_this_frame_rush, color_this_frame_rush = this_frame_rush.x.values.tolist(), this_frame_rush.y.values.tolist(), this_frame_rush.rusher_metric.values.tolist()
    motion_rush[frame, :, :] = [x_this_frame_rush, y_this_frame_rush, color_this_frame_rush]
    
    this_frame_block = block_df.loc[block_df.time_after_snap == time_after].copy()
    x_this_frame_block, y_this_frame_block, color_this_frame_block = this_frame_block.x.values.tolist(), this_frame_block.y.values.tolist(), this_frame_block.x.values.tolist()
    motion_block[frame, :, :] = [x_this_frame_block, y_this_frame_block, color_this_frame_block]

    this_frame_qb = qb_df.loc[block_df.time_after_snap == time_after].copy()
    x_this_frame_qb, y_this_frame_qb, color_this_frame_qb = this_frame_qb.x.values.tolist(), this_frame_qb.y.values.tolist(), this_frame_qb.x.values.tolist()
    motion_qb[frame, :, :] = [x_this_frame_qb, y_this_frame_qb, color_this_frame_qb]
    frame += 1

In [804]:
range_x = [min(np.append(np.append(motion_block[:,0,:], motion_rush[:,0,:]),motion_qb[:,0,:]) ) - 2, max(np.append(np.append(motion_block[:,0,:], motion_rush[:,0,:]),motion_qb[:,0,:])) + 2]
range_y = [min(np.append(np.append(motion_block[0,1,:], motion_rush[:,1,:]),motion_qb[:,1,:]) ) - 2, max(np.append(np.append(motion_block[:,1,:], motion_rush[:,1,:]),motion_qb[:,1,:])) + 2]
color_min, color_max = -0.3, 0.3
vec_mobile = list(zip(motion_rush[:,:,:], motion_block[:,:,:], motion_qb[:,:,:], play_prob[:])) # index is (frame), (rusher or blocker or qb or prob_disrupt), (x or y or metric)
start_frame = [go.Scatter(x=motion_rush[0,0,:], y = motion_rush[0,1,:], 
                            mode = "markers+text", 
                            text = ['Rusher Metric: ' + str(round(metric, 3)) for metric in motion_rush[0,2,:]],
                            textposition="bottom center",
                            name = "Rushers",
                            legendgroup = "Rushers",
                            marker = dict(size=20,color=motion_rush[0,2,:], colorbar=dict(title='Metric'), colorscale='rdylbu_r',showscale=True, cmin=color_min, cmax = color_max)),
               go.Scatter(x=motion_block[0,0,:], y = motion_block[0,1,:], 
                            mode = "markers", 
                            name = "Blockers",
                            legendgroup = "Blockers",
                            marker = dict(size=20, color = 'black'),
                            marker_symbol = "x"  ),
                go.Scatter(x=motion_qb[0,0,:], y = motion_qb[0,1,:], 
                            mode = "markers", 
                            name = "QB",
                            legendgroup = "QB",
                            marker = dict(size=16, color = 'black'),
                            marker_symbol = "diamond"  ),
                go.Scatter(x= np.array(range_x[0] + 4), y = np.array(range_y[1] - 1), 
                            mode = "text",
                            text = ['QB Distruption Probability: ' + str(round(play_prob[0], 3))],
                            textfont = dict(size=17, color = 'black') ) ]

all_frame = [   go.Frame(data = [go.Scatter(x=vec_mobile[i][0][0], y = vec_mobile[i][0][1], 
                                            mode = "markers+text", 
                                            text = ['Rusher Metric: ' + str(round(metric, 3)) for metric in vec_mobile[i][0][2]],
                                            textposition="bottom center",
                                            name = "Rushers",
                                            legendgroup = "Rushers",
                                            marker = dict(size=20, color=vec_mobile[i][0][2],showscale=False, colorscale='rdylbu_r', cmin=color_min, cmax = color_max) ),
                                go.Scatter(x= np.ndarray.flatten(np.array([list(x) for x in np.array(vec_mobile[0:i], dtype = object)[:, 0]])[:, 0]), 
                                            y = np.ndarray.flatten(np.array([list(x) for x in np.array(vec_mobile[0:i], dtype = object)[:, 0]])[:, 1]), 
                                            mode = "markers", 
                                            marker = dict(size=10, color= np.ndarray.flatten(np.array([list(x) for x in np.array(vec_mobile[0:i], dtype = object)[:, 0]])[:, 2] ), 
                                                                colorscale='rdylbu_r',showscale=False, cmin=color_min, cmax = color_max),
                                            marker_symbol = "circle" ),
                                 go.Scatter(x=vec_mobile[i][1][0], y = vec_mobile[i][1][1], 
                                            mode = "markers", 
                                            name = "Blockers",
                                            legendgroup = "Blockers",
                                            marker = dict(size=20, color = 'black'),
                                            marker_symbol = "x" ),
                                go.Scatter(x=vec_mobile[i][2][0], y = vec_mobile[i][2][1], 
                                            mode = "markers", 
                                            name = "QB",
                                            legendgroup = "QB",
                                            marker = dict(size=20, color = 'black'),
                                            marker_symbol = "diamond" ),
                                go.Scatter(x= np.array(range_x[0] + 4), y = np.array(range_y[1] - 1), 
                                                mode = "text",
                                                text = ['QB Distruption Probability: ' + str(round(vec_mobile[i][3], 3))],
                                                textfont = dict(size=17, color = 'black') ) 
                                                ]) 
            for i in range(len(vec_mobile))[1:]]
layout_draw = go.Layout(
        autosize=False,
        width=800,
        height=600,
        plot_bgcolor='green',
        xaxis=dict(title='src freq', range=range_x, autorange=False, tickmode = 'linear', tick0 = round(range_x[0],-1), dtick = 5, gridcolor = 'black'),
        yaxis=dict(range=range_y, autorange=False, gridcolor = 'green', tick0 = round(range_x[0],-1), dtick = 10),
        title= f'Play {play_id}',
        updatemenus=[dict(type="buttons",
                        buttons=[dict(label="Play",
                                    method="animate",
                                    args=[None, {"frame": {"duration": 200, "redraw": False}, "fromcurrent": True, "transition": {"duration": 0}}])])]
    )


In [805]:
from __future__ import annotations
import plotly
import plotly.graph_objects as go
fig = go.Figure(
    data= start_frame,
    layout=layout_draw,
    frames=all_frame
)

fig.update_layout(legend=dict(orientation = "h"))  
fig.show()

In [802]:
# plot player metric over time


AttributeError: 'dict' object has no attribute 'iteritems'

In [962]:





# write function to calculate all player metrics

# load_all_player_metrics_by_play(week)

week = 2


def get_player_metrics(week):
    player_metrics_each_play = {}
    for this_week in range(week):
        all_plays = all_data.search_track_weeks(variables = ['week'], variable_values = [this_week]).playId.unique()
        for play in tqdm.tqdm(all_plays):
            this_play_dat = get_all_player_metrics_on_play(all_data, play_id = play, model = lstm_model) 
            this_play_dat_avgs = {key : sum(this_play_dat.get(key)) / len(this_play_dat.get(key)) for key in this_play_dat.keys()}
            for key in this_play_dat_avgs.keys():
                if (key in player_metrics_each_play.keys()):
                    player_metrics_each_play[key].append(this_play_dat_avgs[key])
                else:
                    player_metrics_each_play.update({key : [ this_play_dat_avgs[key] ]})
    avg_metric_to_this_point = {key : [sum(player_metrics_each_play.get(key)) / len(player_metrics_each_play.get(key)), len(player_metrics_each_play.get(key))] for key in player_metrics_each_play.keys()}
    avg_metric_to_this_point = dict(sorted(avg_metric_to_this_point.items(), reverse = True, key=lambda item: item[1]))

    return(avg_metric_to_this_point)

def get_top_player_metrics(master_track, metric_dict, min_play_n):
    top_n = pd.DataFrame({'nflId' : list(metric_dict.keys()), 'metric' : [item[0] for item in metric_dict.values()], 'n_play' : [item[1] for item in metric_dict.values()]}).astype({'nflId': 'int32'})
    player_df = master_track.player_df.astype({'nflId': 'int32'})
    top_n = top_n.merge(player_df, on = 'nflId', how = 'left')
    top_n = top_n.loc[top_n.n_play >= min_play_n].reset_index(drop = 1)
    top_n['rank'] = top_n.index + 1
    return(top_n)



In [964]:
ranking_df = get_top_player_metrics(master_track = all_data, metric_dict = this, min_play_n = 10)
ranking_df[0:50]

,nflId,metric,n_play,height,weight,birthDate,collegeName,officialPosition,displayName,rank
0,43378,0.058777,25,6-2,305,1993-02-07,South Carolina State,DT,Javon Hargrave,1
1,47800,0.055464,20,6-4,342,1997-11-12,Clemson,DE,Dexter Lawrence,2
2,43356,0.053288,29,6-2,308,1995-04-08,Nebraska,DT,Maliek Collins,3
3,43694,0.052493,11,6-0,340,1992-11-06,Samford,NT,Michael Pierce,4
4,52456,0.052384,12,6-3,253,1997-03-24,Tennessee,DE,Darrell Taylor,5
5,41249,0.052070,29,6-2,252,1991-03-19,Auburn,DE,Dee Ford,6
6,35539,0.050824,17,6-3,273,1987-12-22,Southern California,DE,Everson Griffen,7
7,42497,0.050166,11,6-2,309,1992-09-03,Fresno State,DT,Tyeler Davison,8
8,52582,0.047859,10,6-3,291,1997-04-24,North Carolina State,DT,Larrell Murchison,9
9,52481,0.047156,12,6-4,310,1997-02-01,Ohio State,DT,Davon Hamilton,10


In [954]:
top = get_top_player_metrics(master_track = all_data, metric_dict = this)
top.loc[top.n_play > 5]

,nflId,metric,n_play,height,weight,birthDate,collegeName,officialPosition,displayName,rank
11,52455,0.071654,7,6-3,279,1998-05-11,Auburn,DT,Marlon Davidson,12
15,48032,0.066217,6,6-3,284,1996-05-05,Temple,DE,Michael Dogbe,16
20,53587,0.063581,7,6-3,305,NaN,NaN,DT,Daviyon Nixon,21
26,43378,0.058777,25,6-2,305,1993-02-07,South Carolina State,DT,Javon Hargrave,27
29,41269,0.057523,6,6-3,250,1991-03-26,Brigham Young,OLB,Kyle Van Noy,30
...,...,...,...,...,...,...,...,...,...,...
402,47933,-0.005200,10,6-3,288,1996-09-26,Texas A&amp;M,DE,Kingsley Keke,403
405,46098,-0.005938,10,6-4,291,1996-03-11,Florida,DT,Taven Bryan,406
412,42444,-0.008620,15,6-2,265,1993-08-16,Arkansas,OLB,Trey Flowers,413
414,40171,-0.009713,12,6-4,309,1990-02-21,Samford,DE,Nicholas Williams,415


In [942]:
this = get_player_metrics(2)

0it [00:00, ?it/s]
100%|██████████| 1170/1170 [05:53<00:00,  3.31it/s]


In [943]:
this

{'53547': [0.09664173026879629, 1],
 '52852': [0.09654228885968526, 1],
 '47900': [0.09402313282092413, 1],
 '53957': [0.0926373465494676, 1],
 '45302': [0.08885651024488303, 1],
 '46151': [0.08448825829558902, 3],
 '44926': [0.08384762693532932, 4],
 '43343': [0.07912326597649118, 3],
 '47996': [0.07814658261262454, 1],
 '53529': [0.07568753709395727, 1],
 '44818': [0.07547964172230827, 3],
 '52455': [0.07165372741251232, 7],
 '52607': [0.0697330060104529, 1],
 '43516': [0.06885486543178558, 1],
 '52556': [0.06699898670117062, 5],
 '48032': [0.06621698477528977, 6],
 '43407': [0.0660495372536855, 2],
 '47975': [0.06504235883553823, 5],
 '44893': [0.06428416470686595, 1],
 '42376': [0.06401200691858927, 1],
 '53587': [0.06358078845909663, 7],
 '42116': [0.06326669355233511, 1],
 '46618': [0.062082337020408544, 4],
 '53639': [0.06134926974773407, 1],
 '38559': [0.060805956025918324, 2],
 '42370': [0.060768148608365956, 3],
 '43378': [0.05877747886504414, 25],
 '47846': [0.05802492797374

In [468]:
play_dets = all_data.play_details_df
play_dets['disrupt'] = play_dets.pff_hit + play_dets.pff_hurry + play_dets.pff_sack
print( f'Percent of player play disruptions: {play_dets.disrupt.sum() / len(play_dets.disrupt)}')
team_dets = play_dets.groupby('playId').agg({'disrupt' : [sum]})
team_dets.columns = ["_".join(x) for x in np.array(team_dets.columns).ravel()]
team_dets = team_dets.reset_index()
print( f'Percent of team play disruptions: {len(team_dets.loc[team_dets.disrupt_sum > 0]) / len(team_dets)}')

Percent of player play disruptions: 0.022958343514613237
Percent of team play disruptions: 0.37583265163024426


In [ ]:
# sack / hurry / hit are dependent on both pass coverage and lineman and we only care about linemen. 
# Instead of using sack / hit /hurry, use P(Sack, Hurry, Hit @ time t) given conditions for team
# to evaluate players, need some sort of metric for 'contribution to P(Sack, Hurry, Hit @ time t) on play'
# to do so try to do  or sommin    [ P(Sack, Hurry, Hit @ time t replacing player with rusher average) / P(Sack, Hurry, Hit @ time t) ] - P(Sack, Hurry, Hit @ time t)




# new idea: since there is no team level time of sack
# find model that we can track each player prob of hit/hurry/sack 
# each player route will have a prob of sack at time t. 
# to compare use average route stats in similar situation: note may have to import more data (ie. play type etc)


# also: need to create qb_metric for 'facing rusher' 

In [ ]:
import matplotlib.pyplot as plt
plot_overall_avgs = all_dist_dfs.query("time_after_snap > 0").groupby(by = ['time_after_snap']).mean().reset_index()

plt.title = "Average Rusher Distance From QB by Seconds After Snap"
plt.xlabel = "Seconds After Snap"
plt.plot('time_after_snap',  'rusher_distance_from_qb', data = plot_overall_avgs, linestyle='dashed')
plt.ylabel = "Rusher Distance From QB"
plt.show()

In [ ]:
### want to get groups of players with similar starting positions: conlusion: just use number of blockers on play for now, maybe add in weigt of blocker later

# this_week_starting_pos = all_data.training_data_individual.get('1')
# this_week_starting_pos = this_week_training_individual.loc[this_week_training_individual.time_after_snap == 0, ['playId', 'nflId', 'rusher_distance_from_qb', 'y']]



# gmm_train = this_week_starting_pos.drop(['playId', 'nflId'], axis = 1)
# gmm = mixture.GaussianMixture(n_components=2).fit(gmm_train)
# gmm_labels = gmm.predict(gmm_train).astype(str)

# this_week_starting_pos['starting_pos_class'] = gmm_labels
# this_week_starting_pos.groupby('starting_pos_class').groups.items()
# plot_df = this_week_starting_pos.drop(['playId', 'nflId'], axis = 1).reset_index(drop = 1)
# plot_df.starting_pos_class.unique()
# plot_df

# groups = plot_df.groupby('starting_pos_class')
# # Plot
# fig, ax = plt.subplots()
# ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
# for name, group in groups:
#     ax.plot(group.rusher_distance_from_qb, group.y, marker='o', linestyle='', ms=12, label=name)
# ax.legend()

# plt.show()

# ret_df = this_week_starting_pos.loc[:, ['playId', 'nflId', 'starting_pos_class']]
# ret_df